In [1]:
from importlib import reload
import json
import pretty_midi
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers

In [ ]:
# check to see that the gpu is available
print(tf..device_lib.list_local_devices())

In [2]:
# import modules, including a reload statement so that they can be reimported after a change to the methods 
import modules.midiMethods as midiMethods
reload(midiMethods)
from modules.midiMethods import *

import modules.dataMethods as dataMethods
reload(dataMethods)
from modules.dataMethods import *

import modules.models as models
reload(models)

import modules.mlClasses as mlClasses
reload(mlClasses)


<module 'modules.mlClasses' from '/home/ubuntu/storage/380-music-representation/modules/mlClasses.py'>

## NoteTuple Sequence Generation
This purpose of this notebook is to generate sequences using the NoteTuple/note bin representation.
Weights are loaded into a new model with the same architecture as at training time, but with sequence length of 1, and hidden states retained across batches. The choice of temperature is vitally important on the quality of the new sequences. Low temperatures emphasize more likely events; higher temperatures 'even out' the softmax distribution.
See here for a very good tutorial on text generation: https://www.tensorflow.org/tutorials/text/text_generation

In [5]:
# if using an older version of nb, will need vocab1
vocab1 = {"pitch":88, "shift_M":10, "shift_m":60, "duration_M":18, "duration_m":30, "velocity":32}
vocab2 = {"pitch":88, "shift_M":10, "shift_m":24, "duration_M":13, "duration_m":16, "velocity":16}
hidden_state = 512
lstm_layers = 3
prediction_model = models.create_nbmodel(batch_size=1, stateful=True, hidden_state_size=hidden_state, lstm_layers=lstm_layers, vocab=vocab1)

Model: "3layerLSTM"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(1, 256, 6)]        0                                            
__________________________________________________________________________________________________
lambda_6 (Lambda)               (1, 256, 88)         0           input_2[0][0]                    
__________________________________________________________________________________________________
lambda_7 (Lambda)               (1, 256, 10)         0           input_2[0][0]                    
__________________________________________________________________________________________________
lambda_8 (Lambda)               (1, 256, 60)         0           input_2[0][0]                    
_________________________________________________________________________________________

In [ ]:
# load some weights
prediction_model.load_weights('models/nb/nbmodel21/33-5.30.hdf5')
# prediction_model.load_weights('weights/note_bin/model2_60epochs512state3layer.h5')
# choose an input sequence to start things off with
start = 'spa4'
input_notes = pm2note_bin(pretty_midi.PrettyMIDI(f'midi/starts/{start}.mid'))

In [ ]:
for temperature in np.linspace(0,1,11):
    # we have the option of adjusting temperature for each of the six note attributes independently
    temperatures = [temperature] * 6
    new_seq_length = 400
    new_seq = models.generate_nbmusic(prediction_model, new_seq_length, temperatures, input_notes)
    new_seq_list = [[int(a) for a in note] for note in new_seq]
    # convert to prettymidi object and write to file
    pm = note_bin2pm(new_seq_list)
    pm.write(f'models/nb/nbmodel21/midi/{start}-{new_seq_length}-{temperature:.3f}.mid')